# Voyages API Voyages Like Use Case

A Voyage is defined to be a sequence of Load operations followed by a sequence of Discharges. Users of **Signal Ocean Platform** interface with the concept of a voyage in different levels of detail. For example in the Voyages tab of Vessels Data (https://app.signalocean.com/vessels) users can see  all the operations of a voyage even at jetty level.  
However very often arises the need of conducting an analysis of the voyages for a specific vessel class for a specific time window. This need is accommodated by the  **Voyages Data Dashboard** (https://app.signalocean.com/reportsindex/voyagesdatalive).  

The level of detail provided by the Voyages Data Dashboard has been tailored, having in mind the neccessary information needed to carry out such an analysis without being overwhelmed by the full data provided by Signal Ocean Platform regarding the voyages of the vessels.  

While both ```get_voyages``` and ``get_voyages_flat`` functions of the Signal SDK return the full low level data available, in this example we are going to construct a dataframe that resembles the form of ***Voyages Data Dashboard***

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [1]:
signal_ocean_api_key = '' #replace with your subscription key

In [2]:
from signal_ocean import Connection
from signal_ocean.voyages import VoyagesAPI
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime, timezone
import datetime as dt

In [3]:
connection = Connection(signal_ocean_api_key)
api = VoyagesAPI(connection)

### Get voyages

In [4]:
vlcc_id = 84
date_from = datetime.now(timezone.utc) - timedelta(days=365)

In [ ]:
voyages = api.get_voyages(vessel_class_id=vlcc_id, date_from=date_from)

In [ ]:
voyages = pd.DataFrame(v.__dict__ for v in voyages)
events = pd.DataFrame(e.__dict__ for voyage_events in voyages['events'].dropna() for e in voyage_events)

In [ ]:
def get_load_discharge_events(voyage_events):
    load_event = next((e.__dict__ for e in voyage_events or [] if e.purpose=='Load'), None)
    discharge_event = next((e.__dict__ for e in voyage_events or [] if e.purpose=='Load'), None)
    return pd.Series((load_event, discharge_event))
    
voyages[['load_event','discharge_event']] = voyages['events'].apply(get_load_discharge_events)

In [ ]:
mapping_dict = {'port_name':['load_port','discharge_port'],
                'area_name_level0':['load_port_narrow_area','discharge_port_narrow_area'], 
                'area_name_level2':['load_port_wide_area','discharge_port_wide_area'], 
                'area_name_level1':['load_port_area','discharge_port_area'], 
                'sailing_date':['load_port_sailing_date','discharge_port_sailing_date'], 
                'longitude':['load_port_longitude','discharge_port_longitude'], 
                'latitude':['load_port_latitude','discharge_port_latitude'],
                'event_horizon':['load_horizon','discharge_horizon']}

events = {0:'load_event',1:'discharge_event'}

In [ ]:
for feature,targets in mapping_dict.items():
    for num,target in enumerate(targets):
        voyages[target] = voyages[events[num]].apply(lambda e: e[feature] if isinstance(e,dict) else None)

In [ ]:
def get_last_discharge_port(voyage):
    return next((e.port_name for e in reversed(voyage['events'][:-2] or []) if e.purpose=='Discharge'), None)
    
voyages['last_discharge_port'] = voyages.apply(get_last_discharge_port, axis=1)

In [ ]:
def get_last_3_months_ind(laycan_from):
    if not pd.isnull(laycan_from):
        laycan_from = pd.to_datetime(laycan_from)
        return 1 if ((laycan_from.date()-date.today()).days<4*30) else 0
    else:
        return 0
    
voyages['last_3_months_ind'] = voyages['laycan_from'].apply(get_last_3_months_ind)

In [ ]:
def get_sts_load_ind(load_event):
    return next((1 for d in load_event["event_details"] or [] if d.event_detail_type =='StS'), 0)

def get_sts_discharge_ind(discharge_event):
    return next((1 for d in discharge_event["event_details"] or [] if d.event_detail_type =='StS'), 0)


voyages.loc[voyages.discharge_event.notna(),'sts_discharge_ind'] = \
voyages.loc[voyages.discharge_event.notna(),'discharge_event'].apply(get_sts_discharge_ind)
voyages.loc[voyages.discharge_event.notna(),'sts_load_ind'] = \
voyages.loc[voyages.discharge_event.notna(),'load_event'].apply(get_sts_load_ind)

In [ ]:
vessel_status_dict = {
    1:"Voyage", 2:"Breaking", 3:"Domestic Trade", 4:"FPSO", 5:"FPSO Conversion", 
    6:"Inactive", 7:"Storage Vessel", 9:"Conversion"
}
voyages['vessel_status'] = voyages.vessel_status_id.replace(vessel_status_dict)

In [ ]:
wanted_columns = ['imo',
                  'voyage_number',
                  'vessel_type_id',
                  'vessel_name',
                  'vessel_type',
                  'vessel_class',
                  'trade',
                  'commercial_operator',
                  'charterer',
                  'rate',
                  'rate_type',
                  'cargo_type',
                  'cargo_group',
                  'quantity',
                  'laycan_from',
                  'laycan_to',
                  'fixture_status_id',
                  'fixture_status',
                  'fixture_date',
                  'fixture_is_coa',
                  'fixture_is_hold',
                  'load_port',
                  'discharge_port',
                  'load_port_narrow_area',
                  'discharge_port_narrow_area',
                  'load_port_wide_area',
                  'discharge_port_wide_area',
                  'load_port_area',
                  'discharge_port_area',
                  'load_port_sailing_date',
                  'discharge_port_sailing_date',
                  'load_port_longitude',
                  'discharge_port_longitude',
                  'load_port_latitude',
                  'discharge_port_latitude',
                  'sts_discharge_ind',
                  'sts_load_ind',
                  'load_horizon',
                  'discharge_horizon']

voyages = voyages[wanted_columns]

In [ ]:
voyages = voyages.astype(
                        {
                            'laycan_from':'datetime64[ns]', 
                            'laycan_to':'datetime64[ns]',
                            'fixture_date':'datetime64[ns]',
                            'laycan_to':'datetime64[ns]',
                            'fixture_date':'datetime64[ns]',
                            'load_port_sailing_date':'datetime64[ns]',
                            'discharge_port_sailing_date':'datetime64[ns]'
                        })

In [ ]:
voyages.to_excel('voyages_data.xlsx')